## Modificaciones modelo default

In [32]:
#Script to modify the model iJN1463.xml for FLYCOP simulation
#07/12/20
#Ana del Ramo Galián
from cobra import Reaction
from cobra import Metabolite

import cobra as cb
import cometspy as c
model=cb.io.read_sbml_model("modelManoli.xml")
model

Name,model
Memory address,0x07fb0e9feb670
Number of metabolites,2008
Number of reactions,2625
Number of groups,0
Objective expression,1.0*BiomassKT2440_ME - 1.0*BiomassKT2440_ME_reverse_0e45e
Compartments,"p, c, e"


### Quitamos como fuente de carbono la glucosa

In [33]:
model.reactions.EX_glc_e.bounds=(0,0)
model.reactions.EX_pet_e.bounds=(0,0)

### Añadimos glycoaldehido como fuente de carbono

In [34]:
model.reactions.EX_glycol_e.bounds=(-10,0)

### Añadimos PCA como fuente de carbono

In [35]:
model.reactions.EX_34dhbz_e.bounds=(-5,0)
model.reactions.EX_o2_e.bounds=(-18.5,0)


In [36]:
C80aPHAtpp_reaction=Reaction('C80aPHAtpp')
C80aPHAtpp_reaction.name='C80aPHA transporter periplasm'
C80aPHAtpp_reaction.lower_bound=0
C80aPHAtpp_reaction.upper_bound=1000
c80apha_p = Metabolite('C80aPHA_p', formula='C8H15O3R', name='C80_Medium_chain_length_aliphatic_Polyhydroxyalkanoate_p',compartment='p')
c80apha_e = Metabolite('C80aPHA_e_default', formula='C8H15O3R', name='C80_Medium_chain_length_aliphatic_Polyhydroxyalkanoate_e',compartment='e')
C80aPHAtpp_reaction.add_metabolites({model.metabolites.get_by_id('C80aPHA_c'):-1, c80apha_p:1})
C80aPHAtpp_reaction.reaction

'C80aPHA_c --> C80aPHA_p'

In [37]:
C80aPHAtex_reaction=Reaction('C80aPHAtex')
C80aPHAtex_reaction.name='C80aPHA transporter extracellular'
C80aPHAtex_reaction.lower_bound=0
C80aPHAtex_reaction.upper_bound=1000
C80aPHAtex_reaction.add_metabolites({c80apha_p:-1, c80apha_e:1})
C80aPHAtex_reaction.reaction

'C80aPHA_p --> C80aPHA_e_default'

In [38]:
EX_C80aPHA_e_reaction=Reaction('EX_C80aPHA_e')
EX_C80aPHA_e_reaction.name='C80aPHA exchange'
EX_C80aPHA_e_reaction.lower_bound=0
EX_C80aPHA_e_reaction.upper_bound=1000
EX_C80aPHA_e_reaction.add_metabolites({c80apha_e:-1})
EX_C80aPHA_e_reaction.reaction

'C80aPHA_e_default --> '

In [39]:
model.reactions.get_by_id('DM_C80aPHA').bounds=(0,0)

### Adding the reactions to the model

In [40]:
model.add_reactions([C80aPHAtex_reaction, C80aPHAtpp_reaction, EX_C80aPHA_e_reaction])

## Quitamos las reacciones que dice Juan

In [41]:
for reaction in ['RHACOAE80','PHADPC80','CYSS','NACODA','ALDD2x','ABUTD','ALAR','ALATA_L','FORGLUIH2','ICDHyr','ICL','MALS','MCITL2','OARGDC','ORNCD','PDHbr','PDHcr','PPCSCT','RBK','SUCOAS','UPPN']:
    model.reactions.get_by_id(reaction).bounds=(0,0)

In [42]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
34dhbz_e,EX_34dhbz_e,5,7,81.40%
ca2_e,EX_ca2_e,0.0006952,0,0.00%
cl_e,EX_cl_e,0.0006952,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0004634,0,0.00%
cu2_e,EX_cu2_e,0.0004634,0,0.00%
fe2_e,EX_fe2_e,0.002416,0,0.00%
glycol_e,EX_glycol_e,4,2,18.60%
h_e,EX_h_e,3.461,0,0.00%
k_e,EX_k_e,0.02608,0,0.00%
mg2_e,EX_mg2_e,0.001159,0,0.00%


In [43]:
comets_model=c.model(model)

In [44]:
comets_model.id='defaultModel-3-consumoSeparado'

In [45]:
def ensure_sinks_are_not_exchanges(model):
    rxn_names = ['sink_PHAg']
    for rxn in rxn_names:
        model.reactions.loc[model.reactions.REACTION_NAMES == rxn, 'EXCH'] = False
        model.reactions.loc[model.reactions.REACTION_NAMES == rxn, 'EXCH_IND'] = 0

In [46]:
ensure_sinks_are_not_exchanges(comets_model)

## Saving the model

In [47]:
comets_model.write_comets_model()